# **Building Multi-Agent Systems - David Okpare**


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1ngPQwhKcCd1878oRpW254n9z0yLEia1d?usp=sharing)


## Installation

**Note:** The following cell installs the necessary libraries:

*   `pydantic-ai`: [https://ai.pydantic.dev](https://ai.pydantic.dev)
*   `pydantic`: [https://docs.pydantic.dev/](https://docs.pydantic.dev/)
*   `tavily-python`: [https://docs.tavily.com/welcome](https://docs.tavily.com/welcome)

In [ ]:
!pip install -qqU pydantic-ai pydantic tavily-python

In [ ]:
import os
from typing import Dict, List

from pydantic_ai import Agent
from pydantic import BaseModel
from tavily import TavilyClient

## API Keys

You'll need API keys for the services you'll be using. It's recommended to store these securely in Colab Secrets. For Gemini API keys, you can obtain them from [https://aistudio.google.com/](https://aistudio.google.com/).

For Tavily API keys, you can obtain them from [https://app.tavily.com/home](https://app.tavily.com/home)

1.  **Open Colab Secrets:** Click on the "🔑" icon in the left sidebar.
2.  **Add a new secret:** Click the "+" button to add a new secret.
3.  **Name the secret:** Enter a name for your API key (e.g., `TAVILY_API_KEY`, `GOOGLE_API_KEY`).
4.  **Enter the secret value:** Paste your API key into the value field.
5.  **Save:** Click "Save secret".

Once you've added your API keys to Colab Secrets, you can access them in your notebook like this:

In [ ]:
from google.colab import userdata

# Replace 'YOUR_SECRET_NAME' with the actual name of your secret in Colab Secrets
tavily_api_key = userdata.get('TAVILY_API_KEY')
google_api_key = userdata.get('OPENAI_API_KEY')

# Now you can set these variables in the environment
os.environ['TAVILY_API_KEY'] = tavily_api_key
os.environ['OPENAI_API_KEY'] = google_api_key

### Choosing a model

We'd use Gemini 2.5 Flash. However you can modify the model name to any choice of yours. See [here](https://ai.pydantic.dev/api/models/base/) for all the support model names by Pydantic AI.

In [ ]:
model = "openai:gpt-4.1"

## Configure Multi Agent

This section will cover the implementation of multiple agents and how they interact.

### Writing instructions for Lead agent

This is a prompt to instruct the Lead agent on what to do.

In [ ]:
lead_agent_instruction = """You are a Research Lead Agent responsible for breaking down complex research queries and coordinating subagents to gather comprehensive information.

Your Role:
1. Analyze the user's research query
2. Break it down into 2-4 focused research tasks
3. Deploy subagents sequentially (not in parallel) to avoid rate limits
4. Synthesize findings into a comprehensive final report

Process:
1. First, understand the query scope and complexity
2. Create a simple research plan with 2-4 specific subtasks
3. Use the `run_subagent` tool to assign each subtask to a subagent
4. Wait for each subagent to complete before deploying the next one
5. Carefully examine findings to identify gaps, inconsistencies, or new angles
6. If needed, create additional research tasks to explore different perspectives or fill knowledge gaps
7. Continue iterating until you have sufficient comprehensive information
8. Review all findings and create a final synthesized report

Guidelines:
- Keep research tasks focused and specific
- Deploy subagents one at a time to manage rate limits
- Maximum of 2 follow-up research rounds after initial research to prevent loops
- Ensure each subtask contributes unique value to the overall research
- After each subagent completes, critically assess if the findings answer the research question
- Look for knowledge gaps, conflicting information, or unexplored angles
- Create follow-up tasks to address gaps or explore alternative perspectives
- Stop iterating when you have sufficient coverage OR reach the follow-up limit
- Create a comprehensive final report in structured markdown format
- IMPORTANT: Output ONLY markdown text, no other format or structure
- Follow this EXACT markdown structure:

  # [Clear title that reflects the research scope]

  ## Executive Summary

  [Write an executive summary that captures the essence of your findings - 2-3 sentences]

  ## [Section 1 Title - based on your research findings]

  [Detailed findings for this section]

  ## [Section 2 Title - based on your research findings]

  [Detailed findings for this section]

  ## [Additional sections as needed - typically 2-4 total sections]

  [Continue with logical sections based on your findings]

  ## Key Takeaways

  1. [First key takeaway that directly addresses the user's query]
  2. [Second key takeaway]
  3. [Third key takeaway]
  [Continue with 3-5 total takeaways]

- Do NOT include citations, references, or source lists
- Ensure high information density and be comprehensive yet concise
- Stream your response as you write - don't wait to format everything at once

Tools Available:
- run_subagent: Send specific research tasks to subagents
"""

### Defining lead agent

In [ ]:
lead_agent = Agent(
    model=model,
    instructions=lead_agent_instruction,
    output_type=str
)

In [ ]:
class Task(BaseModel):
    """Task to be assigned to a subagent"""

    description: str
    focus_area: str


class SubagentTasks(BaseModel):
    """List of tasks to run subagents for"""

    tasks: List[Task]

In [ ]:
import asyncio

@lead_agent.tool_plain
async def run_subagent(tasks: SubagentTasks):
    """Run subagents in parallel. Each task should be specific and focused."""
    print(f"🚀 Running {len(tasks.tasks)} subagents...")
    print([{"description": task.description, "focus_area": task.focus_area}
               for task in tasks.tasks])

    results = await asyncio.gather(
        *[
            research_agent.run(
                f"Research Task: {task.description}\nFocus Area: {task.focus_area}",
            )
            for task in tasks.tasks
        ]
    )

    print(f"✅ Completed {len(results)} subagent tasks")

    return [result.output for result in results]

In [ ]:
result = await lead_agent.run("When was the AISoC started and who is the founder?")
print(result.output)

🚀 Running 1 subagents...
[{'description': 'Determine the launch/start year of the AI Summer of Code by examining the official website, program blogs, and announcement pages.', 'focus_area': 'Start date/origins'}]
✅ Completed 1 subagent tasks
🚀 Running 1 subagents...
[{'description': "Identify the founder(s) of AI Summer of Code by examining the program's About page, leadership information, official blog introductions, and credible organizational profiles.", 'focus_area': 'Founders and organizational leadership'}]
✅ Completed 1 subagent tasks
# Origins of the AI Summer of Code

## Executive Summary

The AI Summer of Code was publicly launched in July 2024, marking the beginning of its activities in the artificial intelligence education space. Zion Pibowei is most consistently recognized as the program’s founder and primary organizer, with Anand Ganu also appearing in leadership and possible co-founder roles.

## Launch Date of AI Summer of Code

The program’s public debut occurred in Ju

### Assignment:

The agent should ask for clarification before making a search.